In [29]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import re
import operator
import pandas as pd
import numpy as np
import winsound

In [58]:
US_FILE = 'us'
INDIA_FILE = 'india'
UK_FILE = 'uk'
INDO_FILE = 'indo'

CURR_FILE = US_FILE

data = pd.read_csv('../data/result/freelancer_data_results_'+CURR_FILE+'.csv')

In [59]:
driver = webdriver.Firefox()
driver.implicitly_wait(3)

In [60]:
#learn how to get element
#alternatives:
# - pake xpath detail (tradeoff: kalo halaman beda dikit aja , kek ada elemen diatas, bakal obsolete - harus bikin xpath alternatif)
# - pake kelas khusus (tradeoff: ga semua data punya)
# - a mix of both (lagi dicari. tradeoff: kalo prosesnya terlalu susah, mungkin lama)

all_info_result = []
links = data['link'].values
link_itr = 0

while link_itr in range(len(links)):
    link = links[link_itr]
    print('getting page', link_itr, link)
    try:
        driver.get(link)
        try:
            div_class = 'cfe-aggregates'
            freelancer_info = WebDriverWait(driver, 3).until(
                                                    EC.presence_of_element_located((By.CLASS_NAME, div_class))
                                                )
            
            rate_per_hour_xpath = '//ul/li[1]/div[1]/div/h3/cfe-profile-rate/span/span'
            total_earned_xpath = '//ul/li[2]/h3/span'
            total_jobs_xpath = '//ul/li[3]/h3'
            workhours_xpath = '//ul/li[4]/h3'

            try:
                rate_per_hour = freelancer_info.find_element_by_xpath(rate_per_hour_xpath).get_attribute("innerHTML")
            except:
                rate_per_hour = 'none'
                print('rate per hour get error')
            try:
                total_earned = freelancer_info.find_element_by_xpath(total_earned_xpath).get_attribute("innerHTML")
            except:
                total_earned = 'none'
                print('total earned get error')
            try:
                total_jobs = freelancer_info.find_element_by_xpath(total_jobs_xpath).get_attribute("innerHTML")
            except:
                total_jobs = 'none'
                print('total jobs get error')
            try:
                workhours = freelancer_info.find_element_by_xpath(workhours_xpath).get_attribute("innerHTML")
            except:
                workhours = 'none'
                print('workhours get error')

            all_info_result.append([link, rate_per_hour, total_earned, total_jobs, workhours])
            link_itr += 1
            
        except Exception as e:
            print(e, '- GET INFO DIV ERROR')
            try:
                driver.find_element_by_class_name("air-icon-exclamation")
                print('NEED SIGN IN')
            except:
                try:
                    driver.find_element_by_xpath("/html/body/section/div[2]/div/h1")
                    driver = webdriver.Firefox()
                    driver.implicitly_wait(3)
                    print("new driver")
                except:
                    print(e, '- UNKNOWN ERROR')
                    winsound.Beep(2000, 5000)
            finally:
                all_info_result.append([link, 'none', 'none', 'none', 'none'])
                link_itr += 1
    
    except Exception as e:
        print(e, '- GET PAGE ERROR')
        try:
                driver.find_element_by_xpath("/html/body/section/div[2]/div/h1")
                driver = webdriver.Firefox()
                driver.implicitly_wait(3)
                print("new driver")
        except:
            print(e, "UNKNOWN ERROR, CANNOT ACCESS PAGE")
            winsound.Beep(2000, 5000)
            all_info_result.append([link, 'none', 'none', 'none', 'none'])
            link_itr += 1
                

getting page 0 https://www.upwork.com/o/profiles/users/~010f13f69ef761dda7
getting page 1 https://www.upwork.com/o/profiles/users/~018b5ce44849fd24d7
Message: 
 - GET INFO DIV ERROR
new driver
getting page 2 https://www.upwork.com/o/profiles/users/~012d70da5f7ecbedb2
getting page 3 https://www.upwork.com/o/profiles/users/~0132d5accdad6707f7
getting page 4 https://www.upwork.com/o/profiles/users/~016a1667200a0b8cb0
getting page 5 https://www.upwork.com/o/profiles/users/~01c407f91b187afa7d
getting page 6 https://www.upwork.com/o/profiles/users/~01db74ce3e7227751d
getting page 7 https://www.upwork.com/o/profiles/users/~01fab9a961adfd18a9
getting page 8 https://www.upwork.com/o/profiles/users/~0175c2eb4473099f31
getting page 9 https://www.upwork.com/o/profiles/users/~01d7d072a2ee328490
getting page 10 https://www.upwork.com/o/profiles/users/~0128b1101ef0ea2d98
getting page 11 https://www.upwork.com/o/profiles/users/~010f594802cd7f0912
getting page 12 https://www.upwork.com/o/profiles/users

In [61]:
#assert 1 == 0
saved_data = data.copy()
saved_data['rate_per_hour'] = pd.Series([x[1] for x in all_info_result])
saved_data['total_earned'] = pd.Series([x[2] for x in all_info_result])
saved_data['total_jobs'] = pd.Series([x[3] for x in all_info_result])
saved_data['workhours'] = pd.Series([x[4] for x in all_info_result])
saved_data.head()

,Unnamed: 0,name,link,location,skills,rate_per_hour,total_earned,total_jobs,workhours
0,0,Brian M M.,https://www.upwork.com/o/profiles/users/~010f1...,chesapeake,\nPHP\n<!-- ngIf: isSkillChecked(skill) -->\n;...,\n$40.00\n,\n$9k+\n,\n1\n,\n278\n
1,1,Robert W.,https://www.upwork.com/o/profiles/users/~018b5...,woodbury,Writing\n<!-- ngIf: isSkillChecked(skill) -->,none,none,none,none
2,2,Jami H.,https://www.upwork.com/o/profiles/users/~012d7...,saint joseph,\nGeneral Profile\n;\nBusiness Development\n<!...,\n$20.00\n,\n$60k+\n,\n100\n,"\n4,611\n"
3,3,Michael P.,https://www.upwork.com/o/profiles/users/~0132d...,new york,\nGeneral Profile\n;\nSearch Engine Optimizati...,\n$90.00\n,\n$200k+\n,\n66\n,"\n3,177\n"
4,4,Jack M.,https://www.upwork.com/o/profiles/users/~016a1...,lorain,\nNegotiation\n<!-- ngIf: isSkillChecked(skill...,\n$34.00\n,\n$200k+\n,\n40\n,"\n11,172\n"


In [62]:
saved_data.to_csv("../data/result/freelancer_data_complete_"+CURR_FILE+".csv", index=False)

In [63]:
US_FILE = 'us'
INDIA_FILE = 'india'
UK_FILE = 'uk'
INDO_FILE = 'indo'

CURR_FILE = UK_FILE

data = pd.read_csv('../data/result/freelancer_data_results_'+CURR_FILE+'.csv')

In [111]:
driver = webdriver.Firefox()
driver.implicitly_wait(3)

### impute missing datas

In [112]:
links = saved_data[((saved_data['rate_per_hour'] == 'none') & (saved_data['skills'] != 'invisible'))].loc[1100:,'link'].values

In [113]:
#learn how to get element
#alternatives:
# - pake xpath detail (tradeoff: kalo halaman beda dikit aja , kek ada elemen diatas, bakal obsolete - harus bikin xpath alternatif)
# - pake kelas khusus (tradeoff: ga semua data punya)
# - a mix of both (lagi dicari. tradeoff: kalo prosesnya terlalu susah, mungkin lama)

all_info_result = []
#links = data['link'].values
link_itr = 0

while link_itr in range(len(links)):
    link = links[link_itr]
    print('getting page', link_itr, link)
    try:
        driver.get(link)
        try:
            div_class = 'cfe-aggregates'
            freelancer_info = WebDriverWait(driver, 3).until(
                                                    EC.presence_of_element_located((By.CLASS_NAME, div_class))
                                                )
            
            rate_per_hour_xpath = '//ul/li[1]/div[1]/div/h3/cfe-profile-rate/span/span'
            total_earned_xpath = '//ul/li[2]/h3/span'
            total_jobs_xpath = '//ul/li[3]/h3'
            workhours_xpath = '//ul/li[4]/h3'

            try:
                rate_per_hour = freelancer_info.find_element_by_xpath(rate_per_hour_xpath).get_attribute("innerHTML")
            except:
                rate_per_hour = 'none'
                print('rate per hour get error')
            try:
                total_earned = freelancer_info.find_element_by_xpath(total_earned_xpath).get_attribute("innerHTML")
            except:
                total_earned = 'none'
                print('total earned get error')
            try:
                total_jobs = freelancer_info.find_element_by_xpath(total_jobs_xpath).get_attribute("innerHTML")
            except:
                total_jobs = 'none'
                print('total jobs get error')
            try:
                workhours = freelancer_info.find_element_by_xpath(workhours_xpath).get_attribute("innerHTML")
            except:
                workhours = 'none'
                print('workhours get error')

            all_info_result.append([link, rate_per_hour, total_earned, total_jobs, workhours])
            link_itr += 1
            
        except Exception as e:
            print(e, '- GET INFO DIV ERROR')
            try:
                driver.find_element_by_class_name("air-icon-exclamation")
                print('NEED SIGN IN')
            except:
                try:
                    driver.find_element_by_xpath("/html/body/section/div[2]/div/h1")
                    driver = webdriver.Firefox()
                    driver.implicitly_wait(3)
                    print("new driver")
                except:
                    print(e, '- UNKNOWN ERROR')
                    winsound.Beep(2000, 5000)
            finally:
                all_info_result.append([link, 'none', 'none', 'none', 'none'])
                link_itr += 1
    
    except Exception as e:
        print(e, '- GET PAGE ERROR')
        try:
                driver.find_element_by_xpath("/html/body/section/div[2]/div/h1")
                driver = webdriver.Firefox()
                driver.implicitly_wait(3)
                print("new driver")
        except:
            print(e, "UNKNOWN ERROR, CANNOT ACCESS PAGE")
            winsound.Beep(2000, 5000)
            all_info_result.append([link, 'none', 'none', 'none', 'none'])
            link_itr += 1
                

getting page 0 https://www.upwork.com/o/profiles/users/_~014dffd2c76bf49d92
Message: 
 - GET INFO DIV ERROR
Message: 
 - UNKNOWN ERROR


KeyboardInterrupt: 

In [ ]:
for i in range(len(all_info_result)):
    result_link = all_info_result[i][0]
    saved_data.at[saved_data['link'] == result_link, 'rate_per_hour'] = all_info_result[i][1]
    saved_data.at[saved_data['link'] == result_link, 'total_earned'] = all_info_result[i][2]
    saved_data.at[saved_data['link'] == result_link, 'total_jobs'] = all_info_result[i][3]
    saved_data.at[saved_data['link'] == result_link, 'workhours'] = all_info_result[i][4]

In [76]:
assert 1 == 0
saved_data = data.copy()
saved_data['rate_per_hour'] = "none"
saved_data['total_earned'] = "none"
saved_data['total_jobs'] = "none"
saved_data['workhours'] = "none"

for i in range(len(all_info_result)):
    try:
        if(saved_data.at[i, 'link'] == all_info_result[i][0]):
            saved_data.at[i, 'rate_per_hour'] = all_info_result[i][1]
            saved_data.at[i, 'total_earned'] = all_info_result[i][2]
            saved_data.at[i, 'total_jobs'] = all_info_result[i][3]
            saved_data.at[i, 'workhours'] = all_info_result[i][4]
    except Exception as e:
        print("Error:", e, "at", i)
        continue

,name,link,location,skills,rate_per_hour,total_earned,total_jobs,workhours
0,Jill S.,https://www.upwork.com/o/profiles/users/_~0133...,london,\nEmail Handling\n<!-- ngIf: isSkillChecked(sk...,\n$15.00\n,\n$10k+\n,\n14\n,"\n1,656\n"
1,Agnieszka F.,https://www.upwork.com/o/profiles/users/_~0103...,sheffield,invisible,\n$5.00\n,\n$60k+\n,\n5\n,"\n10,434\n"
2,Zofia R.,https://www.upwork.com/o/profiles/users/_~0106...,sutton coldfield,\nWriting\n<!-- ngIf: isSkillChecked(skill) --...,\n$16.00\n,\n$4k+\n,\n3\n,\n230\n
3,Gabie L.,https://www.upwork.com/o/profiles/users/_~01e8...,pirbright,\nGeneral Profile\n;\nContent Writing\n<!-- ng...,\n$25.00\n,\n$6k+\n,\n7\n,\n260\n
4,Arin D.,https://www.upwork.com/o/profiles/users/_~01fe...,london,\nGoogle Ads\n<!-- ngIf: isSkillChecked(skill)...,\n$125.00\n,\n$10k+\n,\n17\n,\n243\n
5,Ottilia K.,https://www.upwork.com/o/profiles/users/_~01bd...,devon,\nVirtual Assistant\n<!-- ngIf: isSkillChecked...,\n$24.00\n,\n$90k+\n,\n14\n,"\n5,335\n"
6,Jake W.,https://www.upwork.com/o/profiles/users/_~016e...,bristol,\nGeneral Profile\n;\nEditing &amp; Proofreadi...,\n$75.00\n,\n$80k+\n,\n169\n,"\n1,241\n"
7,Agnes P.,https://www.upwork.com/o/profiles/users/_~012f...,winkleigh,\nGeneral Profile\n;\nTranslation\n<!-- ngIf: ...,\n$15.00\n,\n$10k+\n,\n72\n,"\n1,283\n"
8,Benjamin N.,https://www.upwork.com/o/profiles/users/_~0112...,bristol,\nCopywriting\n<!-- ngIf: isSkillChecked(skill...,\n$38.50\n,\n$90k+\n,\n306\n,\n804\n
9,Gary C.,https://www.upwork.com/o/profiles/users/_~0183...,london,\nGeneral Profile\n;\nSocial Media Marketing\n...,\n$45.00\n,\n$30k+\n,\n18\n,\n356\n


In [80]:
temp = pd.DataFrame([[1,2,3], [2,3,4], [3,4,5]], columns=['A','B','C'])
temp.at[temp['B']==3, 'B'] = 9
temp

,A,B,C
0,1,2,3
1,2,9,4
2,3,4,5


In [ ]:
saved_data.to_csv("../data/result/freelancer_data_complete_"+CURR_FILE+".csv", index=False)

In [49]:
assert 1 == 0
driver.get('https://www.upwork.com/o/profiles/users/~01c979ddcba7bd74bd/')
freelancer_skills = WebDriverWait(driver, 3).until(
                                        EC.presence_of_element_located((By.XPATH, '//ul/li[4]/h3'))
                                    )